# Predicting GBP/USD Currency Exchange Rate Using RNN (POC 1)
## Step 1: Data Loading and Preprocessing


The historical data contains information about the volume traded and the corresponding GBP/USD exchange rate at different timestamps.

1) Import necessary libraries including numpy, pandas, scikit-learn, and Keras.
2) Load Data: Load the historical data from a CSV file into a pandas DataFrame.

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

# get the historical data of GBPUSD from 2021.01.01 to 2024.04.11
data = pd.read_csv("GBPUSD_Hourly_Ask_2021.01.01_2024.04.11.csv") 
print(data.shape)
data.head(10)

We’ll predict the **closing prices,** which can be accessed in the **Close column.**



In [ ]:
# Selecting 'Volume' as feature and 'Close' as target
features = data['Volume'].values.reshape(-1, 1)
target = data['Close'].values

# Normalizing the data
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target.reshape(-1, 1))

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target_scaled, test_size=0.2, random_state=42)

# Reshaping input to be [samples, time steps, features] which is required for RNN
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

## Step 2: Model Building

In [ ]:
# Build the RNN model
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(1, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

print(model.summary()) 


## Step 3: Model Training

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Predicting using the trained model
predictions = model.predict(X_test)

## Step 4: Model Evaluation

In [ ]:
# Rescaling the predicted values back to original scale
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

# Calculating the mean squared error and mean absolute percentage error
mse = mean_squared_error(y_test_rescaled, predictions_rescaled)
mape = mean_absolute_percentage_error(y_test_rescaled, predictions_rescaled)

# Creating a table for results
results_df = pd.DataFrame({
    "Metric": ["Mean Squared Error", "Mean Absolute Percentage Error"],
    "Value": [mse, mape]
})

# Displaying the results in a DataFrame
print(results_df)